<a href="https://colab.research.google.com/github/arvindsamriya/Text-Classification-Sentiment-Analysis/blob/master/amazon_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [155]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Description 
The objective of this project is to detect the racist and sexist comments from amazon reviews or does a review contain any hate speech in it,we will classify these reviews.
We will use a Training sample to build a model and use that model on a test dataset to predict the labels on that dataset and find the accuracy of our model.

# Introduction
In this whole process we will use the amazon review 'txt' file to build the system and in this file we have 10000 examples, We 
will follow the following components to get the result-
#### -Dataset Preparation
#### -Feature Extraction
#### - Training Model

In [156]:
##libraries
import pandas as pd
import numpy as np
from sklearn import model_selection,preprocessing,svm,metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB 
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

## Dataset Preparation

In [157]:
#We will convert the raw data into a meaning full and informative dataset
review_data=pd.read_csv('/content/drive/MyDrive/Data Sets/amazon review/review.txt',sep=('\n'),header=None)
print(review_data)
#review_data.iloc[:,0]=review_data.iloc[:,0].str.lower()
review_data['label']=''
review_data['Text']=''
texts=[]
for i in review_data.index:
    body=str(review_data.iloc[i,0]).split()
    review_data['label'][i]=body[0]
    review_data['Text'][i]=" ".join(body[1:])
review_data.head()

                                                      0
0     __label__2 Stuning even for the non-gamer: Thi...
1     __label__2 The best soundtrack ever to anythin...
2     __label__2 Amazing!: This soundtrack is my fav...
3     __label__2 Excellent Soundtrack: I truly like ...
4     __label__2 Remember, Pull Your Jaw Off The Flo...
...                                                 ...
9995  __label__2 A revelation of life in small town ...
9996  __label__2 Great biography of a very interesti...
9997  __label__1 Interesting Subject; Poor Presentat...
9998  __label__1 Don't buy: The box looked used and ...
9999  __label__2 Beautiful Pen and Fast Delivery.: T...

[10000 rows x 1 columns]


,0,label,Text
0,__label__2 Stuning even for the non-gamer: Thi...,__label__2,Stuning even for the non-gamer: This sound tra...
1,__label__2 The best soundtrack ever to anythin...,__label__2,The best soundtrack ever to anything.: I'm rea...
2,__label__2 Amazing!: This soundtrack is my fav...,__label__2,Amazing!: This soundtrack is my favorite music...
3,__label__2 Excellent Soundtrack: I truly like ...,__label__2,Excellent Soundtrack: I truly like this soundt...
4,"__label__2 Remember, Pull Your Jaw Off The Flo...",__label__2,"Remember, Pull Your Jaw Off The Floor After He..."


## Cleaning Text
- Removing Stopwords
- removing puntuations
- removing all words that contain numbers
- lemmatizer
- include only Verb,noun and Adjectives



In [158]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords
import string
import re
exclude = set(string.punctuation)
stopword=set(stopwords.words('english'))
def clean_text(doc):
  text1=[i for i in nltk.word_tokenize(doc.lower()) if i not in stopword]
  text2=[i for i in text1 if i not in exclude]
  text3=[w for w in text2 if re.search('^\D',w)]
  text4=" ".join([nltk.WordNetLemmatizer().lemmatize(i) for i in text3])
  #text5=" ".join([w for (w,pt) in nltk.p.os_tag(text4) if pt=='JJ' or pt=='NN' or pt=='JJR' or pt=='JJS']) 
  return text4
clean_text=review_data['Text'].apply(lambda x:clean_text(x))
clean_text
#clean_text("Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0       stuning even non-gamer sound track beautiful p...
1       best soundtrack ever anything 'm reading lot r...
2       amazing soundtrack favorite music time hand in...
3       excellent soundtrack truly like soundtrack enj...
4       remember pull jaw floor hearing 've played gam...
                              ...                        
9995    revelation life small town america early thoug...
9996    great biography interesting journalist biograp...
9997    interesting subject poor presentation 'd hard-...
9998    n't buy box looked used obviously new tried co...
9999    beautiful pen fast delivery pen shipped prompt...
Name: Text, Length: 10000, dtype: object

## Feature Extraction using TFidf Vectorization

In [159]:
x_train,x_test,y_train,y_test= model_selection.train_test_split(clean_text,review_data['label'])
#split the data into test and training datasets
encoder = preprocessing.LabelEncoder()
y_train= encoder.fit_transform(y_train)
y_test= encoder.fit_transform(y_test)
#we transform y_train and y_test into the binary class according to the labels.
tfidf=TfidfVectorizer(min_df=20,max_df=0.90,ngram_range=(1,2)).fit(x_train)
#we have use stopwords like he,are,is etc these words don't bring any feature to our dataset.
#min_df removes words which are only present in less than 10 documents in the whole x_train.
#max_df removes words which are present in 99% of the x_train documents.
x_train=tfidf.transform(x_train)
x_test=tfidf.transform(x_test)
#feature extraction is completed here.
#tfidf.get_feature_names() 

In [160]:
tfidf_df=pd.DataFrame(x_train.todense(),columns=tfidf.get_feature_names())
tfidf_df.head()

,ability,able,absolute,absolutely,ac,accent,access,account,accurate,across,act,acting,action,action movie,actor,actress,actual,actually,adam,adam sandler,adapter,add,added,addition,admit,adult,advanced,adventure,advertised,advice,affair,afraid,again,age,ago,agree,ahead,ai,air,air mattress,...,would given,would good,would highly,would like,would make,would never,would nice,would read,would recommend,would work,wow,write,writer,writes,writing,writing style,written,written book,wrong,wrote,xp,yeah,year,year ago,year later,year old,year solitude,yes,yet,york,you,young,young woman,younger,yr,zen,zero,zero star,zeta,zeta jones
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.093033,0.0,0.0,0.0,0.0,0.14612,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.331118,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Different ML Models and Model Selection
We have extracted features and labels dataset, and now we will apply different ML models on the training dataset and calculate the accuracy for each of them.
- Naive Bayes
- Logistic Regression
- Support Vector Machine(SVM)
- Random Forest Model

In [161]:
#Naive Bayes
gridval={'alpha':[0.001,.01,.1,1,10]}
model=MultinomialNB()
grid=GridSearchCV(model,param_grid=gridval,scoring='accuracy')
grid.fit(x_train,y_train)
pred= grid.predict(x_test)
accuracy_mnb=accuracy_score(y_test,pred)*100

In [162]:
#Logistic Regression
gridval={'C':[0.0001,0.001,.01,.1,1,10]}
model=LogisticRegression()
grid=GridSearchCV(model,param_grid=gridval,scoring='accuracy')
grid.fit(x_train,y_train)
pred= grid.predict(x_test)
accuracy_lrg=accuracy_score(y_test,pred)*100

In [163]:
#Support Vector Machine (SVM)
#gridval={'gamma':[0.0001,0.001,.01,.1,1,10],'C':[0.0001,0.001,.01,.1,1,10]}
model=SVC(kernel='rbf')
#grid=GridSearchCV(model,param_grid=gridval,scoring='accuracy')
#grid.fit(x_train,y_train)
#pred= grid.predict(x_test)
model.fit(x_train,y_train)
pred=model.predict(x_test)
accuracy_svm=accuracy_score(y_test,pred)*100


In [164]:
# Random Forest Model(RFM)
model=RandomForestClassifier()
#gridval={'n_estimators':[10,100,200,300,400,500]}
#grid=GridSearchCV(model,param_grid=gridval,scoring='accuracy')
#grid.fit(x_train,y_train)
model.fit(x_train,y_train)
#pred= grid.predict(x_test)
pred=model.predict(x_test)
accuracy_rfm=accuracy_score(y_test,pred)*100

In [165]:
#Result
#Now we have accuracy of different models-
print('Accuracies of Different Models\nNaive Bayer: {}\nLogistic Regression: {}\nSupport Vector Machine(SVM):{}\nRandom Forest Model: {}\n '.format(accuracy_mnb,accuracy_lrg,accuracy_svm,accuracy_rfm))

Accuracies of Different Models
Naive Bayer: 85.28
Logistic Regression: 86.2
Support Vector Machine(SVM):86.04
Random Forest Model: 83.91999999999999
 
